In [132]:
import pandas as pd
import numpy as np
import spacy
import nltk
import string
import re

from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from langdetect import detect

import warnings

nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /Users/sheng-
[nltk_data]     tselin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/sheng-
[nltk_data]     tselin/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/sheng-
[nltk_data]     tselin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [133]:
df = pd.read_json("indeed_jobs_20251004_170227.json")
jd = df["description"].to_frame()
jd

,description
0,"Job Description\nRole: In t his role, you will..."
1,Are you fueled by a passion for computer visio...
2,Job Description\nRole: As a n AI/ML Engineer o...
3,Job Description\nHybrid This role is categoriz...
4,Job Description\nHybrid This role is categoriz...
5,"Job Description\nOur AI Research team, reporti..."
6,Job Description\nJob Description\nHybrid This ...
7,"Job Description\nOur AI Research team, reporti..."
8,"Job Description\nRole: In t his role, you will..."
9,Job Description\nAdvantest is seeking a versat...


In [134]:
def clean_text(raw):

    try:
        raw = raw.replace("\r", "")
        raw = raw.replace("\n", " ")
        raw = re.sub(r"[\[\]\(\)\{\}]", " ", raw)
        raw = re.sub("\bnull\b", "")
        
        clean_html = re.compile("<.*?>")
        clean_text = re.sub(clean_html, " ", raw)
        clear_text = re.sub(r"\s+", " ", raw).strip()
        clean_text = re.sub(" , ", ", ", clean_text)
        clean_text = clean_text.replace(".,", ".")

    except:
        clean_text = raw

    return clean_text

In [135]:
jd["description"] = jd["description"].apply(clean_text)

In [136]:
jd

,description
0,"Job Description Role: In t his role, you will ..."
1,Are you fueled by a passion for computer visio...
2,Job Description Role: As a n AI/ML Engineer on...
3,Job Description Hybrid This role is categorize...
4,Job Description Hybrid This role is categorize...
5,"Job Description Our AI Research team, reportin..."
6,Job Description Job Description Hybrid This ro...
7,"Job Description Our AI Research team, reportin..."
8,"Job Description Role: In t his role, you will ..."
9,Job Description Advantest is seeking a versati...


In [137]:
jd["lower_d"] = jd["description"].str.lower()

tokenizer = RegexpTokenizer(r"\w+")
jd["word_tokenized"] = jd["lower_d"].apply(lambda row: tokenizer.tokenize(row))
jd["sentence_tokenized"] = jd["description"].apply(lambda row: sent_tokenize(row))


In [138]:
jd

,description,lower_d,word_tokenized,sentence_tokenized
0,"Job Description Role: In t his role, you will ...","job description role: in t his role, you will ...","[job, description, role, in, t, his, role, you...","[Job Description Role: In t his role, you will..."
1,Are you fueled by a passion for computer visio...,are you fueled by a passion for computer visio...,"[are, you, fueled, by, a, passion, for, comput...",[Are you fueled by a passion for computer visi...
2,Job Description Role: As a n AI/ML Engineer on...,job description role: as a n ai/ml engineer on...,"[job, description, role, as, a, n, ai, ml, eng...",[Job Description Role: As a n AI/ML Engineer o...
3,Job Description Hybrid This role is categorize...,job description hybrid this role is categorize...,"[job, description, hybrid, this, role, is, cat...",[Job Description Hybrid This role is categoriz...
4,Job Description Hybrid This role is categorize...,job description hybrid this role is categorize...,"[job, description, hybrid, this, role, is, cat...",[Job Description Hybrid This role is categoriz...
5,"Job Description Our AI Research team, reportin...","job description our ai research team, reportin...","[job, description, our, ai, research, team, re...","[Job Description Our AI Research team, reporti..."
6,Job Description Job Description Hybrid This ro...,job description job description hybrid this ro...,"[job, description, job, description, hybrid, t...",[Job Description Job Description Hybrid This r...
7,"Job Description Our AI Research team, reportin...","job description our ai research team, reportin...","[job, description, our, ai, research, team, re...","[Job Description Our AI Research team, reporti..."
8,"Job Description Role: In t his role, you will ...","job description role: in t his role, you will ...","[job, description, role, in, t, his, role, you...","[Job Description Role: In t his role, you will..."
9,Job Description Advantest is seeking a versati...,job description advantest is seeking a versati...,"[job, description, advantest, is, seeking, a, ...",[Job Description Advantest is seeking a versat...


In [143]:
def rm_stopwords(text):
    stop_words = set(stopwords.words("english"))
    cleaned = [word for word in text if word not in stop_words]
    return cleaned

In [144]:
jd["cleaned_stopword"] = jd["word_tokenized"].apply(rm_stopwords)

In [146]:
jd["cleaned_stopword"].head()

0    [job, description, role, role, forefront, deve...
1    [fueled, passion, computer, vision, machine, l...
2    [job, description, role, n, ai, ml, engineer, ...
3    [job, description, hybrid, role, categorized, ...
4    [job, description, hybrid, role, categorized, ...
Name: cleaned_stopword, dtype: object

In [147]:
def lemming(text):
    lemmatizer = WordNetLemmatizer()
    lemmed = [lemmatizer.lemmatize(word) for word in text]
    return lemmed

In [148]:
jd["cleaned_lemmed"] = jd["cleaned_stopword"].apply(lemming)

In [ ]:
jd.head(10)

,description,lower_d,word_tokenized,sentence_tokenized,cleaned_stopword,cleaned_lemmed
0,"Job Description Role: In t his role, you will ...","job description role: in t his role, you will ...","[job, description, role, in, t, his, role, you...","[Job Description Role: In t his role, you will...","[job, description, role, role, forefront, deve...","[job, description, role, role, forefront, deve..."
1,Are you fueled by a passion for computer visio...,are you fueled by a passion for computer visio...,"[are, you, fueled, by, a, passion, for, comput...",[Are you fueled by a passion for computer visi...,"[fueled, passion, computer, vision, machine, l...","[fueled, passion, computer, vision, machine, l..."
2,Job Description Role: As a n AI/ML Engineer on...,job description role: as a n ai/ml engineer on...,"[job, description, role, as, a, n, ai, ml, eng...",[Job Description Role: As a n AI/ML Engineer o...,"[job, description, role, n, ai, ml, engineer, ...","[job, description, role, n, ai, ml, engineer, ..."
3,Job Description Hybrid This role is categorize...,job description hybrid this role is categorize...,"[job, description, hybrid, this, role, is, cat...",[Job Description Hybrid This role is categoriz...,"[job, description, hybrid, role, categorized, ...","[job, description, hybrid, role, categorized, ..."
4,Job Description Hybrid This role is categorize...,job description hybrid this role is categorize...,"[job, description, hybrid, this, role, is, cat...",[Job Description Hybrid This role is categoriz...,"[job, description, hybrid, role, categorized, ...","[job, description, hybrid, role, categorized, ..."
5,"Job Description Our AI Research team, reportin...","job description our ai research team, reportin...","[job, description, our, ai, research, team, re...","[Job Description Our AI Research team, reporti...","[job, description, ai, research, team, reporti...","[job, description, ai, research, team, reporti..."
6,Job Description Job Description Hybrid This ro...,job description job description hybrid this ro...,"[job, description, job, description, hybrid, t...",[Job Description Job Description Hybrid This r...,"[job, description, job, description, hybrid, r...","[job, description, job, description, hybrid, r..."
7,"Job Description Our AI Research team, reportin...","job description our ai research team, reportin...","[job, description, our, ai, research, team, re...","[Job Description Our AI Research team, reporti...","[job, description, ai, research, team, reporti...","[job, description, ai, research, team, reporti..."
8,"Job Description Role: In t his role, you will ...","job description role: in t his role, you will ...","[job, description, role, in, t, his, role, you...","[Job Description Role: In t his role, you will...","[job, description, role, role, forefront, deve...","[job, description, role, role, forefront, deve..."
9,Job Description Advantest is seeking a versati...,job description advantest is seeking a versati...,"[job, description, advantest, is, seeking, a, ...",[Job Description Advantest is seeking a versat...,"[job, description, advantest, seeking, versati...","[job, description, advantest, seeking, versati..."


In [155]:
from keybert import KeyBERT

In [156]:
model = KeyBERT()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [181]:
model.extract_keywords(df["description"], keyphrase_ngram_range = (1, 3))

[[('robotics foundation', 0.5529),
  ('robotics foundation models', 0.5104),
  ('advancements ai frameworks', 0.509),
  ('driving robotics work', 0.4969),
  ('latest advancements ai', 0.4707)],
 [('innovate ai making', 0.5536),
  ('innovate ai', 0.5448),
  ('ai innovation ring', 0.5394),
  ('ai making', 0.5388),
  ('computer vision machine', 0.5285)],
 [('ana lytics frameworks', 0.5517),
  ('ana lytics framework', 0.5451),
  ('scalable automation tools', 0.5155),
  ('ana lytical frameworks', 0.5114),
  ('metrics frameworks team', 0.4979)],
 [('ai compute platform', 0.6181),
  ('ml compute platform', 0.6056),
  ('ai infrastructure gm', 0.5912),
  ('platform ai compute', 0.587),
  ('compute platform ai', 0.5854)],
 [('ai compute platforms', 0.6338),
  ('platform ai compute', 0.6078),
  ('compute platforms ml', 0.5769),
  ('gpus optimizations inference', 0.5703),
  ('ai infrastructure gm', 0.5662)],
 [('ai gm manufacturing', 0.6689),
  ('gm ai research', 0.6477),
  ('gm ai', 0.6221),
  ('